# Example - Convert dataset to raster (GeoTiff)

Often, it is desirable to take a variable (band) out of your dataset and export it to a raster.
This is possible with the `rio.to_raster()`method. It does most of the work for you so you don't
have to.

Note: The `rio.to_raster()` method only works on a 2-dimensional or 3-dimensional `xarray.DataArray` or a 2-dimensional `xarray.Dataset`.

API Reference:

- DataArray: [rio.to_raster()](../rioxarray.rst#rioxarray.raster_array.RasterArray.to_raster)
- Dataset: [rio.to_raster()](../rioxarray.rst#rioxarray.raster_dataset.RasterDataset.to_raster)


In [1]:
import rioxarray

See docs for [rioxarray.open_rasterio](../rioxarray.rst#rioxarray-open-rasterio)

In [2]:
rds = rioxarray.open_rasterio(
    "../../test/test_data/input/PLANET_SCOPE_3D.nc",
)
rds

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
    spatial_ref  int32 0
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...
Attributes:
    coordinates:  spatial_ref

## Converting Dataset to raster

Dataset: [rio.to_raster()](../rioxarray.rst#rioxarray.raster_dataset.RasterDataset.to_raster)

In [3]:
# note how one time slice was selected on export to make the dataset 2D
rds.isel(time=0).rio.to_raster("planet_scope.tif")

In [4]:
!rio info planet_scope.tif

{"bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": ["blue", "green"], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["nodata"], ["nodata"]], "nodata": NaN, "res": [3.0, 3.0], "shape": [10, 10], "tiled": false, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}


## Converting DataArray to raster

DataArray: [rio.to_raster()](../rioxarray.rst#rioxarray.raster_array.RasterArray.to_raster)

In [5]:
# note how selecting one variable allowed for multiple time steps in a single raster
rds.green.rio.to_raster("planet_scope_green.tif")

In [6]:
!rio info planet_scope_green.tif

{"bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": ["green", "green"], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["nodata"], ["nodata"]], "nodata": NaN, "res": [3.0, 3.0], "shape": [10, 10], "tiled": false, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}


## Converting DataArray to raster in a different format
Example here, an ER Mapper grid.
Look at gdal for possible formats that you can use to write to.

In [4]:
# you will get a two file raster: the .ers file with the metdata and the data with no extension
rds.blue.rio.to_raster("planet_scope_green.ers", driver="ERS")

## Change the compression of the raster and explicitly make it a Geotiff

In [5]:
rds.blue.rio.to_raster("planet_scope_green_LZW_compression.tif", driver="GTiff", compress="LZW")

## Change the basic datatype of the raster (in this example, also saving space going to 32 bit)

In [7]:
rds.blue.dtype

dtype('float64')

In [ ]:
rds.blue.astype('float32').rio.to_raster("planet_scope_green_LZW_compression.tif", driver="GTiff", compress="LZW")

## Memory efficient raster writing

Useful for reading and writing larger rasters to disk.

Note: This will increase the time it takes to generate the raster.

Also see:

- [Reading and Writing with Dask](dask_read_write.ipynb)
- [Reading COGs in Parallel](read-locks.ipynb)

In [7]:
rds = rioxarray.open_rasterio(
    "../../test/test_data/input/PLANET_SCOPE_3D.nc",
    lock=False,  # disable internal caching
    cache=False,  # don't keep data loaded in memory. pull from disk every time
)

rds.green.rio.to_raster(
    "planet_scope_tiled.tif",
    tiled=True,  # GDAL: By default striped TIFF files are created. This option can be used to force creation of tiled TIFF files.
    windowed=True,  # rioxarray: read & write one window at a time
) 

In [8]:
!rio info planet_scope_tiled.tif

{"blockxsize": 256, "blockysize": 256, "bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": ["green", "green"], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["nodata"], ["nodata"]], "nodata": NaN, "res": [3.0, 3.0], "shape": [10, 10], "tiled": true, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}
